<a href="https://colab.research.google.com/github/TeAmP0is0N/Hostility-Detection/blob/master/models/BERT_Baseline_Failed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import sys
sys.path.append('/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/Code')

In [ ]:
# pip install flair

In [ ]:
import private
import re
import tweepy
import difflib
import pandas as pd
import numpy as np
import pickle
import time
import copy
import matplotlib.pyplot as plt
from datetime import date
from collections import Counter 

# from flair.data import Sentence
# from flair.embeddings import BertEmbeddings
# embeddings = BertEmbeddings('bert-base-multilingual-uncased', layers='-1') # use only last layers
# import torch


In [ ]:
# sentence = Sentence("Hello hi how are you?")
# embeddings.embed(sentence)
# for token in sentence:
#     z = token.embedding.size()[0]
#     w = torch.zeros(0,z)
#     # storing word Embeddings of each word in a sentence #
#     w = torch.cat((w,token.embedding.view(-1,z)),0)
# # storing sentence Embeddings (mean of embeddings of all words)   #
# s = torch.zeros(0,z)
# s = torch.cat((s, w.mean(dim = 0).view(-1, z)),0)
# s = s.numpy()
# text_embeddings = s
# print(type(s))
# print(s.shape)

In [ ]:
pd.options.display.max_colwidth = 0

In [ ]:
train = pd.read_csv("/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Constraint 21/Hostility Approach/Complete Dataset/Final Processed Dataset/70-10-20/train.csv")
valid = pd.read_csv("/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Constraint 21/Hostility Approach/Complete Dataset/Final Processed Dataset/70-10-20/valid.csv")
test  = pd.read_csv("/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Constraint 21/Hostility Approach/Complete Dataset/Final Processed Dataset/70-10-20/test.csv")

In [ ]:
print(train.shape)
print(valid.shape)
print(test.shape)

(5728, 3)
(811, 3)
(1653, 3)


In [ ]:
train = train.dropna()
valid = valid.dropna()
test  = test.dropna()

print(train.shape)
print(valid.shape)
print(test.shape)

(5728, 3)
(811, 3)
(1653, 3)


In [ ]:
train.head(5)

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive"
1,2,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs",non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE : https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे. https://t.co/4e6lysg0VR,non-hostile


In [ ]:
def preprocess(text):
    text = text.lower()
    url = r'http\S+'
    text = re.sub(url, ' ', text, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    text =  emoji.sub(r'', text)
    text = ' '.join([word[1:] if word[0] == '#' else word for word in text.split()])

    text = text.split()
    exclude_char_list = ['@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL']
    text = [word for word in text if all(ch not in word for ch in exclude_char_list)]
    text = ' '.join(text)
    return text

In [ ]:
def assign_labels(text):

    text   = text.split(',')
    labels = [0,0,0,0,0]
    

    for item in text:
        if item=='defamation':
            labels[0] = 1
            break
        
    for item in text:
        if item=='fake':
            labels[1] = 1
            break

    for item in text:
        if item=='hate':
            labels[2] = 1
            break

    for item in text:
        if item=='offensive':
            labels[3] = 1
            break
        
    for item in text:
        if item=='non-hostile':
            labels[4] = 1
            break    
    
    return labels

In [ ]:
def prepare_data(df):
    
    for i in range(len(df)):    
        df.at[i,'Post'] = preprocess(df['Post'][i])
        df.at[i, 'Labels Set'] = assign_labels(df['Labels Set'][i])

    return df


In [ ]:
train = prepare_data(train)
valid = prepare_data(valid)
test  = prepare_data(test)

print(train.shape)
print(valid.shape)
print(test.shape)

(5728, 3)
(811, 3)
(1653, 3)


In [ ]:
train = train.append(valid)
train = train.drop(columns='Unique ID', axis =1)
train = train.reset_index(drop=True)

test = test.drop(columns='Unique ID', axis =1)
test = test.reset_index(drop=True)


print(train)
print(test)

                                                                                                                                                                                                                                                                                                                                                                                 Post       Labels Set
0     मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम                                                                                                                                                                                                                                                                                   [0, 0, 1, 1, 0]
1     सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.                                                                                                    

In [ ]:
def create_binary_data(df, category, name_of_category):
    total_data = []
    for i in range(len(df)):
        sample = []
        text = df['Post'][i]
        labels = df['Labels Set'][i]

        sample.append(text)
        if(labels[category]==1):
            sample.append(1)
        else:
            sample.append(0)

        total_data.append(sample)
    
    total_data = pd.DataFrame(total_data, columns = ['Post',name_of_category])
    return total_data

In [ ]:
defamation_train = create_binary_data(train,0,"Defamation")
defamation_test  = create_binary_data(test,0,"Defamation")

fake_train = create_binary_data(train,1,"Fake")
fake_test  = create_binary_data(test,1,"Fake")

hate_train = create_binary_data(train,2,"Hate")
hate_test  = create_binary_data(test,2,"Hate")

offensive_train = create_binary_data(train,3,"Offensive")
offensive_test  = create_binary_data(test,3,"Offensive")

non_hostile_train = create_binary_data(train,4,"Non-Hostile")
non_hostile_test  = create_binary_data(test,4,"Non-Hostile")

In [ ]:
defamation_train

,Post,Defamation
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,0
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.",0
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर live :",0
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,1
4,unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी - 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी - 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत - कंटेनमेंट जोन में कोई छूट नहीं - सिनेमाहॉल अभी बंद रहेंगे - 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.,0
...,...,...
6534,चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से गद्दारी को भौंकने तैयार ड्रग्स के नशे में पूरी तरह खोखले हो चुके दाऊद के गुलाम देशद्रोही गद्दार फिल्मी भांडों के भोंकने को भी मीडिया ब्रेकिंग न्यूज़ बना इन दो कौड़ी के बिकाऊ नशेड़ी गद्दार कुत्तों को हीरो बना देती है देश की विडंबना,0
6535,आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने का सहारा हो गया नज़रों की इस मारक क्षमता को क्या कहगे।,1
6536,"औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोना संक्रमण, पिछले 6 दिनों में मिले इतने नए मरीज bihar corona aurangabad",0
6537,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कलम का धमाल और तीन बत्ती के जग्गू का नाना संग असली बवाल bioscope angaar shashilalnair dimplekapadia",0


In [ ]:
# pip install simpletransformers

In [ ]:
# from simpletransformers.classification import ClassificationModel
# import pandas as pd
# import logging

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)



wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
# def train_model(train_df):

#     model = ClassificationModel('bert', 'bert-base-uncased', args={'reprocess_input_data': True, 'overwrite_output_dir': True,'num_train_epochs': 10})# You can set class weights by using the optional weight argument
#     model.train_model(train_df)

#     return model

In [ ]:
# defamation_model = train_model(defamation_train)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


In [ ]:
# a,b = defamation_model.predict(defamation_test['Post'])
# print(sum(a))
# print(b)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



0
[[ 1.10839844 -1.30078125]
 [ 1.10839844 -1.30078125]
 [ 1.10839844 -1.30078125]
 ...
 [ 1.10839844 -1.30078125]
 [ 1.10839844 -1.30078125]
 [ 1.109375   -1.30078125]]


In [ ]:
# from simpletransformers.classification import MultiLabelClassificationModel
# import pandas as pd
# import logging


# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns, a 'text' and a 'labels' column. The `labels` column should contain multi-hot encoded lists.
# # train_data = [['Example sentence 1 for multilabel classification.', [1, 1, 1, 1, 0, 1]], ['This is another example sentence. ', [0, 1, 1, 0, 0, 0]]]
# train_df = pd.DataFrame(train, columns=['text', 'labels'])

# # eval_data = [['Example eval sentence for multilabel classification.', [1, 1, 1, 1, 0, 1]], ['Example eval senntence belonging to class 2', [0, 1, 1, 0, 0, 0]]]
# eval_df = pd.DataFrame(valid)

# # Create a MultiLabelClassificationModel
# model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=5, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 10})
# # You can set class weights by using the optional weight argument
# print(train_df.head())

# # Train the model
# model.train_model(train_df)

# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)
# print(result)
# print(model_outputs)



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

                                                                                                                                                                                                                                                                                    text           labels
0  मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम                                                                                                                                                                                         [0, 0, 1, 1, 0]
1  सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.                                                                                                                                                                       [0, 0, 0, 0, 1]
2  सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए '

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:851: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'LRAP': 0.7771544047129747, 'eval_loss': 0.3895602839104101}
[[0.00338936 0.00557709 0.00345612 0.00356483 0.9921875 ]
 [0.01799011 0.03004456 0.01467133 0.01525116 0.90869141]
 [0.19226074 0.62792969 0.18518066 0.09552002 0.01882935]
 ...
 [0.00327301 0.00598145 0.00319672 0.00338936 0.99121094]
 [0.00298119 0.00901794 0.00266266 0.00291252 0.98730469]
 [0.00332451 0.00579834 0.00331116 0.00345612 0.99169922]]


In [ ]:
# predictions, raw_outputs = model.predict([sample[0] for sample in test])
# print(predictions)
# print(raw_outputs)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[[0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 0, 1], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 1], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1], [0, 0, 0, 1,

In [ ]:
test[0]

['कीस की को रोजगार चाहिए फिर नहीं कहना रोजगार नहीं मिलता है 20 करोड को रोजगार दे दिया है वह भी मात्र 6 साँल में चार साल अभी बाकी है और हर साल दो करोड़ रोजगार देने का ही वादा था 10 साल में देना था 20 करोड को लोगो को रोजगार जो मात्र 6 साल में लक्ष्य को प्राप्त करने वाली पहली सरकार है',
 [1, 1, 0, 1, 0]]

In [ ]:
# from simpletransformers.classification import MultiLabelClassificationModel
# import pandas as pd
# import logging


# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Train and Evaluation data needs to be in a Pandas Dataframe containing at least two columns, a 'text' and a 'labels' column. The `labels` column should contain multi-hot encoded lists.
# train_data = [['Example sentence 1 for multilabel classification.', [1, 1, 1, 1, 0, 1]], ['This is another example sentence. ', [0, 1, 1, 0, 0, 0]]]
# train_df = pd.DataFrame(train_data, columns=['text', 'labels'])

# eval_data = [['Example eval sentence for multilabel classification.', [1, 1, 1, 1, 0, 1]], ['Example eval senntence belonging to class 2', [0, 1, 1, 0, 0, 0]]]
# eval_df = pd.DataFrame(eval_data)

# # Create a MultiLabelClassificationModel
# model = MultiLabelClassificationModel('roberta', 'roberta-base', num_labels=6, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 5})
# # You can set class weights by using the optional weight argument
# print(train_df.head())

# # Train the model
# model.train_model(train_df)

# # Evaluate the model
# result, model_outputs, wrong_predictions = model.eval_model(eval_df)
# print(result)
# print(model_outputs)

# predictions, raw_outputs = model.predict(['This thing is entirely different from the other thing. '])
# print(predictions)
# print(raw_outputs)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
INFO:filelock:Lock 140326011312000 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock


INFO:filelock:Lock 140326011312000 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:filelock:Lock 140326011313400 acquired on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock


INFO:filelock:Lock 140326011313400 released on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultiLabelSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'clas

INFO:filelock:Lock 140325997785160 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
INFO:filelock:Lock 140325997785160 acquired on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:851: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



{'LRAP': 0.855, 'eval_loss': 0.6719530820846558}
[[0.45361328 0.56005859 0.52099609 0.50292969 0.50830078 0.50683594]
 [0.45068359 0.55322266 0.51025391 0.49487305 0.51660156 0.49926758]]



[[0, 1, 1, 0, 1, 1]]
[[0.44555664 0.55957031 0.51074219 0.49560547 0.51025391 0.50146484]]


In [ ]:


# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_data = [['Example sentence belonging to class 1', 1], ['Example sentence belonging to class 0', 0]]
train_df = pd.DataFrame(train_data)

eval_data = [['Example eval sentence belonging to class 1', 1], ['Example eval sentence belonging to class 0', 0]]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-uncased', args={'reprocess_input_data': True, 'overwrite_output_dir': True,})# You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(model_outputs)
print(wrong_predictions)

INFO:filelock:Lock 140283580663120 acquired on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock


INFO:filelock:Lock 140283580663120 released on /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
INFO:filelock:Lock 140281364835352 acquired on /root/.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock


INFO:filelock:Lock 140281364835352 released on /root/.cache/torch/transformers/f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

INFO:filelock:Lock 140280544334904 released on /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:851: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'tp': 1, 'tn': 0, 'fp': 1, 'fn': 0, 'eval_loss': 0.8800287246704102}



{'mcc': 0.0, 'tp': 1, 'tn': 0, 'fp': 1, 'fn': 0, 'eval_loss': 0.8800287246704102}
[[ 0.0534668   1.015625  ]
 [-0.01947021  1.14550781]]


In [ ]:
outputs = model.predict(["heyllo haiii","sdfsdgad  sfsd  s","asdf sd fsda fsd sd "])
print(outputs[0])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.



[1 1 1]
